#Preparing and Preprocessing Dataset

####Importing datasets to local environment and installing additional required libraries

In [1]:
!pip install dnspython tldextract
!wget https://raw.githubusercontent.com/mitchellkrogza/Phishing.Database/master/ALL-phishing-links.tar.gz
# !wget https://phishstats.info/phish_score.csv
# !wget https://data.mendeley.com/public-files/datasets/72ptz43s9v/files/26197eb8-15bc-4e06-a269-aa10ddc286f0/file_downloaded

     |████████████████████████████████| 241 kB 5.4 MB/s 
     |████████████████████████████████| 87 kB 5.7 MB/s 
--2021-11-15 04:48:27--  https://raw.githubusercontent.com/mitchellkrogza/Phishing.Database/master/ALL-phishing-links.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16156630 (15M) [application/octet-stream]
Saving to: ‘ALL-phishing-links.tar.gz’

ALL-phishing-links. 100%[===================>]  15.41M  --.-KB/s    in 0.1s    

2021-11-15 04:48:28 (114 MB/s) - ‘ALL-phishing-links.tar.gz’ saved [16156630/16156630]



####Importing *all* required libraries as per the requirements

In [2]:
#For dataset's preprocessing
import tarfile
import pandas as pd

#For feature engineering
import tldextract 
import urllib.parse as parser
import dns.resolver as dns

####Importing and preprocessing the dataset

In [3]:
def github_data_retrieval(filename):  
  file = tarfile.open(filename+'.tar.gz')
  file.extractall("./github_data")
  file.close()
  df = pd.read_csv('./github_data/'+filename+'.txt', delimiter = "\n")
  f = open("./github_data/"+filename+".txt", "r")
  l = f.readlines()
  new_list = []
  z = []
  for i in l:
    temp = i.replace("\n","").split(",")
    if len(temp) == 8:
      new_list.append(temp)
    else:
      # print(temp)
      # input()
      z.append(temp)
  #NEED TO CLEAN "temp"
  # print(type(new_list[-1]))
  return pd.DataFrame(new_list[:-1],columns=new_list[-1]),z

In [4]:
dataset1,_ = github_data_retrieval("ALL-phishing-links")

In [5]:
# df = pd.read_csv("./phish_score.csv",skiprows=9,names=["Date","Score","URL","IP"])
dataset2 = pd.read_csv("./malicious_url_train_dataset.csv",index_col=0)

FileNotFoundError: ignored

#Feature Engineering

##Features Engineering Functions

In [ ]:
def url_length(column):
  return pd.Series([len(i) for i in column])

In [ ]:
def at_present(column):
  return pd.Series([ 1 if i.find("@") == -1 else -1 for i in column ])

In [ ]:
def dash_present(column):
  return pd.Series([ 1 if i.find("-") == -1 else -1 for i in column ])

In [ ]:
def redirect_present(column):
  flags = []
  for i in column:
    if i.find("https://") != -1 :
      i.replace("https://","")
    if i.find("http://") != -1 :
      i.replace("https://","")
    if i.find("//") != -1:
      flags.append(-1)
    else:
      flags.append(1)  
  return pd.Series(flags)

In [ ]:
# def len_sub_domain(column):
#   flags = []
#   for i in column:
#     i.replace("https://","")
#     i.replace("http://","")
#     l = i.split("/")[0]
#     print(l,len(l))
#     flags.append(len(l))
#   return pd.Series(flags)

In [ ]:
def check_dns_record(column):
  flags = []
  # input()
  for i in column:
    try:
      dns.resolve(i)
      # print("in")
      # input()
      flags.append(1)
    except:
      # print("out")
      flags.append(-1)
  return pd.Series(flags)

In [ ]:
def check_domain_length(column):
  flags = []
  for i in column:
    parsed_url = parser.urlparse(i)
    print(i,parsed_url.netloc)
    # input()
    flags.append(len(parsed_url.netloc))
  return pd.Series(flags)

In [ ]:
def no_of_subdomains(column):
  flags = []
  for i in column:
    parsed_url = tldextract.extract(i)
    flags.append(len(parsed_url.subdomain.split(".")))
  return pd.Series(flags)

##Feature Engineering Execution

We extract these features from the prepared datasets

In [ ]:
a = check_dns_record(dataset2.url)

In [ ]:
a = check_domain_length(dataset2.url)

KeyboardInterrupt: ignored

#Prediction